# Coursera capstone project
In this notebook, I will be working on the applied data science capstone project.

## Import section

In [5]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Data processing section